<a href="https://colab.research.google.com/github/ShivaKondapalli/NLPPyTorch/blob/master/Text_Generation_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this Notebook, we shall generate text from **Francis Ford Coppola's Masterpiece Apocalypse Now**. We will use 
an **LSTM network** with 2 layers to build  a **Languague model**.  We'll surely have bots writing scripts for us in the 
future!

First, let's get a high level walkthrough of what the model: **LSTM** , and an exciting techinque in NLP called LANGAUGE MODELLING are all about.  We will then see how how the two are used to generate text. 

The code reference for this, almost verbatim the same is : "[link text](https://github.com/LeanManager/NLP-PyTorch/blob/master/Character-Level%20LSTM%20with%20PyTorch.ipynb)" 


**LSTM**

Install Pypdf2 to extract data from pdf file.

In [0]:
pip install PyPDF2

     |████████████████████████████████| 81kB 6.4MB/s 
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [0]:
# import all libraries

import torch
from PyPDF2 import PdfFileReader
import re
import numpy as np
import torch.nn.functional as F
import torch.nn as nn

In [0]:
if torch.cuda.is_available():
  print('The power of the GPU is with you!')
else:
  print('Trainingi your model forever!')
  
  
torch.cuda.get_device_name()

The power of the GPU is with you!


'Tesla K80'

In [0]:
path = "Apocalypse_Now.pdf"

file_object = open(path, 'rb')

def read_data(file_object):

    data = PdfFileReader(file_object)

    if data.isEncrypted:
        data.decrypt('')

    return data


def extract_all_data(data):

    text = ''

    for i in range(1, data.numPages):

        pageobj = data.getPage(i)

        text += pageobj.extractText()

    return text
  

# Extracting file object
file_object = read_data("Apocalypse_Now.pdf")

# reading data into data_list, every page of our pdf is a string
text = extract_all_data(file_object)  # length = 138, numofPages.

In [0]:
def clean_text(text):
    """function to convert text to lowercase, retain alphabets, replace hyphenation with space"""

    text = text.replace('-', '')
    text = re.sub(r'\d+', '', text)
    text = text.translate({ord(i): None for i in '!"?();´'})
    text = text.replace('--', '')

    return text


Clean_Text = clean_text(text)

chars = tuple(set(Clean_Text))

int2char = dict(enumerate(chars))

char2int = {w: i for i, w in int2char.items()}

# each word in tokens converted to an integer.
encoded = np.array([char2int[char] for char in Clean_Text])
print(encoded)
print(encoded.shape)

[19 19 37 ... 36  3  0]
(158684,)


In [0]:
def one_hot_encode(arr, n_labels):

    # Initialize the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot
 
ch = np.array([[char2int['w']]])
print(ch)
print(ch.shape)

[[44]]
(1, 1)


In [0]:
def get_batches(arr, n_seqs, n_steps):

    batch_size = n_seqs * n_steps

    # total number of characters in our batch, in vision, number of images in a batch
    # is batch size, 64, 32 and so on. total # of batches is the length of training set/batch_size.

    n_batches = len(arr)//batch_size  # total num of batches to make from encoded array.

    # following is total characters to keep from encoded array to have full batches which are evenly spaced.

    # number of batches * characters per batch will give the number of characters to slice form our encoded array.
    arr = arr[:n_batches * batch_size]
    # print(arr)
    # print(arr.shape)

    # we will be splitting this array enc into N number of batches each having seq_len number of characters.

    arr = arr.reshape((n_seqs, -1))
    # print(enc)
    # print(enc.shape)

    # sq_len * number of batches, this should be equal to the second dimension of our array.
    # print(40 * 396) # product of this with n_seqs_per_batch gives total number of characters.

    # 10 * 40 window 10 * 15480

    # we now need to split our features

    for n in range(0, arr.shape[1], n_steps):

        x = arr[:, n:n+n_steps]

        y = np.zeros_like(x)

        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]

        yield x, y
        
batches = get_batches(encoded, 10, 40)
x, y = next(batches)
# five rows,
print(x[:5])
print('')
print(y[:5])

[[19 19 37 35 38 31 36 13  8 26 19  5 20  8 31 37 19 19 36  8 35 26 40 19
   0  8 20  3 38 43 19 32 46 19 52 23  9 18 19 23]
 [11 51 51 19 46  1 34 55 39 46 19 52 23  9 18 19  9  1 34 43 32 55 23 42
  19 16 32 51 32 43 11  9 18 29  6 24 53 23 55 19]
 [47 43 23  9 46  6 47 34 51 51 19 34 47 19 11 55 39 19  1 14 14 19 43  1
  19 43 53 23 19 46 32 39 23 29 19 24 44  1 19 11]
 [46 45  1 47 23 29 19 21 32 51  4  1  9 23 19 43 11 50 23 46 19 32 43 19
  11 55 39  6  1 47 23 55 46 19 43 53 23 19 17  1]
 [38 26 26  8 35  0 31  1 46 43 51 18 19 14  9  1 16 19 43 53 23 19 32 16
  47  9 32 55 43 46 19 44 53 23 55 19 43 53 23 18]]

[[19 37 35 38 31 36 13  8 26 19  5 20  8 31 37 19 19 36  8 35 26 40 19  0
   8 20  3 38 43 19 32 46 19 52 23  9 18 19 23 11]
 [51 51 19 46  1 34 55 39 46 19 52 23  9 18 19  9  1 34 43 32 55 23 42 19
  16 32 51 32 43 11  9 18 29  6 24 53 23 55 19 11]
 [43 23  9 46  6 47 34 51 51 19 34 47 19 11 55 39 19  1 14 14 19 43  1 19
  43 53 23 19 46 32 39 23 29 19 24 44  1 19 11  9

In [0]:
class CharLSTM(nn.Module):

    def __init__(self, chars, hidden_size, n_layers=2, drop_out=0.5, lr= 0.001):

        super(CharLSTM, self).__init__()

        # Set all the hyperparameters of your network

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.drop_out = drop_out
        self.lr = lr

        # set vocabulary and get indices for these
        self.chars = chars
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {w : i for i, w in self.int2char.items()}

        # define the lstm network, this outputs the next char and cell state and hidden state
        self.lstm = nn.LSTM(input_size=len(self.chars), hidden_size=hidden_size, num_layers=n_layers, dropout=drop_out,
                       batch_first=True)
        # add dropout
        self.drop_out = nn.Dropout(drop_out)

        self.fc = nn.Linear(hidden_size, len(chars))

        self.init_weights()

    def forward(self, x, h_0):
        """compute current and hidden units, stack outputs and pass it to linear layer"""

        x, (h, c) = self.lstm(x, h_0)

        x = self.drop_out(x)

        # reshape x: stack the predicted and hidden state outputs
        # cause fully-connected.
        x = x.view(x.size()[0]*x.size()[1], self.hidden_size)

        x = self.fc(x)

        return x, (h, c)

    def predict(self, char, h=None, cuda=False, top_k=None):
        """given a character, predict the next character in the sequence."""
        
        if cuda:
          self.cuda()
        else:
          self.cpu()
            

        # Initialize hidden state
        if h is None:
            h = self.init_hidden(1)

        # get the integer of character.
        x = np.array([[self.char2int[char]]])

        # one_hot_encode
        x = one_hot_encode(x, len(self.chars))

        # convet to tensor
        one_hot = torch.from_numpy(x)
        
        if cuda:
          one_hot = one_hot.cuda()

        # create a tuple of the hidden state
        # this is what LSTM expects
        h = tuple([each.data for each in h])
        out, h = self.forward(one_hot, h)

        # Prob distribution over all the characters
        probs = F.softmax(out, dim=1).data

        # convert back prob to cpu if model was
        # set to gpu
        
        if cuda:
          probs = probs.cpu()

        # if top number of preds to get
        # wasn't pass, take the distribution over whole character length
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            probs, top_ch = probs.topk(top_k)
            top_ch = top_ch.numpy().squeeze()

        # reduce dims of size 1
        probs = probs.numpy().squeeze()

        char = np.random.choice(top_ch, p=probs/probs.sum())

        return self.int2char[char], h

    def init_weights(self):

        initrange = 0.1

        self.fc.bias.data.fill_(0)
        self.fc.weight.data.uniform_(-1, 1)

    def init_hidden(self, n_seqs):

        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.hidden_size).zero_(),
                weight.new(self.n_layers, n_seqs, self.hidden_size).zero_())

In [0]:
def train(model, data, epochs=10, n_seqs=10, n_steps=40, lr=0.001, clip=5, val_frac= 0.1, cuda=False,
          print_every=10):

    """ model: the model to b trained
        data: the data on which we train
        epochs: number of epochs to train for
        n_seqs" number of sequences in our batch
        n_steps: time step for each sequence
        lr: learning rate
        clip: value used to clip the network gradient to prevent exploding gradeint.
        val_frac: the fraction of data used for validation
        print_every: the number of seconds for which we print out model statistics
    """

    # change model to train mode
    model.train()

    # define optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    

    # trin and validation split
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
      model.cuda()
    
    counter = 0
    n_chars = len(model.chars)

    # loop over epochs
    for epoch in range(epochs):

        # initialize hidden layer of the model
        h = model.init_hidden(n_seqs)

        # loop over batches
        for x, y in get_batches(data, n_seqs, n_steps):

            counter += 1

            # one hot encode
            x = one_hot_encode(x, n_chars)

            # convert to tensors
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            # move inputs and targets to cuda
            
            inputs, targets = inputs.cuda(), targets.cuda()
                
            # New hidden state being created to prevented backpropogating through the 
            # entire history
            h = tuple([each.data for each in h])

            # zero out gradient to prevent accumulation
            model.zero_grad()

            # get output and hidden
            out, h = model.forward(inputs, h)
            loss = criterion(out, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))

            # backpropogate loss
            loss.backward()

            # use gradient clipping to prevent exploding gradient
            nn.utils.clip_grad_norm_(model.parameters(), clip)

            # take a step in the los surface
            optimizer.step()

            if counter % print_every ==0:

                # initilize hidden state for validation
                val_hidden = model.init_hidden(n_seqs)
                val_losses = []

                for x, y in get_batches(val_data, n_seqs, n_steps):

                    x = one_hot_encode(x, n_chars)

                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    val_hidden = tuple([each.data for each in val_hidden])
                    
                    inputs, targets = x, y
                    
                    if cuda:
                      inputs, targets = inputs.cuda(), targets.cuda()
                      
                    out, val_hidden = model.forward(inputs, val_hidden)

                    val_loss = criterion(out, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))

                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(epoch+1, epochs),
                     "Step: {}...".format(counter),
                     "Loss: {:.4f}...".format(loss.item()),
                     "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [0]:
model = CharLSTM(chars, hidden_size=512, n_layers=2)
model

CharLSTM(
  (lstm): LSTM(58, 512, num_layers=2, batch_first=True, dropout=0.5)
  (drop_out): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=58, bias=True)
)

In [0]:
n_seqs, n_steps = 128, 100
train(model, encoded, epochs=25, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=10)

Epoch: 1/25... Step: 10... Loss: 3.4311... Val Loss: 3.4529
Epoch: 2/25... Step: 20... Loss: 3.2848... Val Loss: 3.3369
Epoch: 3/25... Step: 30... Loss: 3.2019... Val Loss: 3.2092
Epoch: 4/25... Step: 40... Loss: 3.0406... Val Loss: 3.0688
Epoch: 5/25... Step: 50... Loss: 2.9227... Val Loss: 2.9252
Epoch: 6/25... Step: 60... Loss: 2.7988... Val Loss: 2.8039
Epoch: 7/25... Step: 70... Loss: 2.7149... Val Loss: 2.7215
Epoch: 8/25... Step: 80... Loss: 2.6249... Val Loss: 2.6663
Epoch: 9/25... Step: 90... Loss: 2.5707... Val Loss: 2.6180
Epoch: 10/25... Step: 100... Loss: 2.5670... Val Loss: 2.5751
Epoch: 10/25... Step: 110... Loss: 2.4933... Val Loss: 2.5237
Epoch: 11/25... Step: 120... Loss: 2.3985... Val Loss: 2.4964
Epoch: 12/25... Step: 130... Loss: 2.3454... Val Loss: 2.4608
Epoch: 13/25... Step: 140... Loss: 2.3752... Val Loss: 2.4306
Epoch: 14/25... Step: 150... Loss: 2.3003... Val Loss: 2.3989
Epoch: 15/25... Step: 160... Loss: 2.3240... Val Loss: 2.3680
Epoch: 16/25... Step: 170.

In [0]:
model_name = "lstm.net"

checkpoint = {'n_hidden': model.hidden_size, 
             'n_layers': model.n_layers, 
             'droput': model.drop_out, 
             'state_dict': model.state_dict(), 
             'toekns': model.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def sample(model, size, prime='The', top_k=None, cuda=False):
        
    if cuda:
        model.cuda()
    else:
        model.cpu()

    model.eval()
    
    # Prime characeters are starting points of some data one can use
    chars = [ch for ch in prime]
    
    h = model.init_hidden(1)
    
    for ch in prime:
        char, h = model.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        
        char, h = model.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [0]:
print(sample(model, 2000, prime='Horror', top_k=5, cuda=True))

Horrors  hom to beat the thryer then whers in the starding ane
the runglys,, becting in hempant over houghing the range sood at hat sting thing of hought.
 thous a fat the some the backer is itsind in
thing he mean. It thar though the bort.     continuing thing
to his oun they.CHEFThes     continuing
 that in the bage.CHEF     continuing
his this and aroust here and the strenting.         hermore.CHee ant as the bott outhathes a frachandeds        sheme and then toware a boods to hin arears that ane a the sonder somen on a tring the string.   M.D. VIEW ON WILLARDWellard.The Coptain shim some thing the rear.
They were  starteds the this his of his are
tring.CHOINEWhit's a the bart tries. Theyerants in out a frothong. HE  ON SHE  THe thing sellien and ano and at and cante the brich.    M.V.COAN T    continuing out the strening inther thas as to brow thengede the sand burt stickes to ser a to thit.CHIEF hat's to be the ridg take atowing.       cherrige ant the beer the stand tancer. It's 

In [0]:
print(sample(model, 5000, prime="doors", top_k=5, cuda=True))

doors. Thing he man shat an       crepting in hissen and the sileter the riget the mant. The sore the seep. They aromald bucking the sand to santont      chon the
rands of here.WILLARDI the strom in.KILGOREThat his sold of the bound thene with the rivile..     conthinging.The Canto andstall stats.Willard, stars ant a micet in it andit  a ment to the sontowhere this are to and beang. I cond of to met and the somento and saling.WILLARDWhat the rear. The raverered, boting the te the tance thingen thing he conds our the singlo anothes ard the bund off ham ant to the racking..Willard to sties the right hore shoth the sear into his songing and a back of ancathe houd on the tore. I'd gottar int of here the sald thing the ristrom his a dank aloont. The Callare.Willard looks at theme armen arouther oun that the conce. We the munted with the clacher stomess thime and and bot this the wat in han bereans an the crach the stare sing the thater the boat.  the mont inte this bouts and sond of the rom